In [ ]:
import numpy as np
from keras import Model
from keras.layers import Layer
from keras.layers import Input, Dense, SimpleRNN
import tensorflow as tf                                # Tensorflow META Package: tf
import tensorflow.keras.activations as tfa             # Tensorflow -> Keras -> Activations = tfa
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.metrics import mean_squared_error

In [ ]:
# Prepare the dataset
# Generate the Fibonacci Sequence
def get_fib_seq(n, scale_data=True):
    # Get the Fibonacci sequence
    seq = np.zeros(n)
    fib_n1 = 0.0
    fib_n = 1.0
    for i in range(n):
        seq[i] = fib_n1 + fib_n
        fib_n1 = fib_n
        fib_n = seq[i]
    scaler = []
    if scale_data:
        scaler = MinMaxScaler(feature_range=(0, 1))
        seq = np.reshape(seq, (n, 1))
        seq = scaler.fit_transform(seq).flatten()
    return seq, scaler

fib_seq = get_fib_seq(10, False)[0]
print(fib_seq)

[ 1.  2.  3.  5.  8. 13. 21. 34. 55. 89.]


In [ ]:
# Get Compile Training Examples and Target values
def get_fib_XY(total_fib_numbers, time_steps, train_percent, scale_data=True):
    dat, scaler = get_fib_seq(total_fib_numbers, scale_data)
    Y_ind = np.arange(time_steps, len(dat), 1)
    Y = dat[Y_ind]
    rows_x = len(Y)
    X = dat[0:rows_x]
    for i in range(time_steps-1):
        temp = dat[i+1:rows_x+i+1]
        X = np.column_stack((X, temp))
    # random permutation with fixed seed
    rand = np.random.RandomState(seed=13)
    idx = rand.permutation(rows_x)
    split = int(train_percent*rows_x)
    train_ind = idx[0:split]
    test_ind = idx[split:]
    trainX = X[train_ind]
    trainY = Y[train_ind]
    testX = X[test_ind]
    testY = Y[test_ind]
    trainX = np.reshape(trainX, (len(trainX), time_steps, 1))
    testX = np.reshape(testX, (len(testX), time_steps, 1))
    return trainX, trainY, testX, testY, scaler

trainX, trainY, testX, testY, scaler = get_fib_XY(12, 3, 0.7, False)
print('trainX = ', trainX)
print('trainY = ', trainY)

trainX =  [[[ 8.]
  [13.]
  [21.]]

 [[ 5.]
  [ 8.]
  [13.]]

 [[ 2.]
  [ 3.]
  [ 5.]]

 [[13.]
  [21.]
  [34.]]

 [[21.]
  [34.]
  [55.]]

 [[34.]
  [55.]
  [89.]]]
trainY =  [ 34.  21.   8.  55.  89. 144.]


In [ ]:
# Predefined parameters
time_steps = 20
hidden_units = 2
epochs = 30

# Function to create Simple RNN
def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(hidden_units, input_shape=input_shape, activation=activation[0]))
    model.add(Dense(units=dense_units, activation=activation[1]))
    model.compile(loss='mse', optimizer='adam')
    return model

# 1. Generate dataset
trainX, trainY, testX, testY, scaler = get_fib_XY(1200, time_steps, 0.7)

# 2. Model Created: model_RNN
model_RNN = create_RNN(hidden_units=hidden_units, dense_units=1, input_shape=(time_steps,1),
activation=['tanh', 'tanh'])

# 2.1 Visualize
# model_RNN.summary()

# 3. Train the network
model_RNN.fit(trainX, trainY, epochs=epochs, batch_size=1, verbose=2)

# 4. Evaluate model
train_mse = model_RNN.evaluate(trainX, trainY)
test_mse = model_RNN.evaluate(testX, testY)

# 5. Print error
print("Train set MSE = ", train_mse)
print("Test set MSE = ", test_mse)

Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


826/826 - 4s - 5ms/step - loss: 6.2155e-04
Epoch 2/30
826/826 - 4s - 5ms/step - loss: 4.7133e-04
Epoch 3/30
826/826 - 2s - 3ms/step - loss: 3.6114e-04
Epoch 4/30
826/826 - 3s - 3ms/step - loss: 2.6951e-04
Epoch 5/30
826/826 - 3s - 3ms/step - loss: 2.1609e-04
Epoch 6/30
826/826 - 5s - 6ms/step - loss: 1.4797e-04
Epoch 7/30
826/826 - 2s - 3ms/step - loss: 1.1758e-04
Epoch 8/30
826/826 - 2s - 3ms/step - loss: 8.9288e-05
Epoch 9/30
826/826 - 3s - 4ms/step - loss: 7.7293e-05
Epoch 10/30
826/826 - 5s - 6ms/step - loss: 7.0602e-05
Epoch 11/30
826/826 - 2s - 3ms/step - loss: 6.8970e-05
Epoch 12/30
826/826 - 2s - 2ms/step - loss: 7.1050e-05
Epoch 13/30
826/826 - 3s - 4ms/step - loss: 6.8680e-05
Epoch 14/30
826/826 - 5s - 6ms/step - loss: 6.7824e-05
Epoch 15/30
826/826 - 2s - 2ms/step - loss: 6.4311e-05
Epoch 16/30
826/826 - 3s - 3ms/step - loss: 6.7137e-05
Epoch 17/30
826/826 - 2s - 3ms/step - loss: 6.2809e-05
Epoch 18/30
826/826 - 2s - 3ms/step - loss: 6.4777e-05
Epoch 19/30
826/826 - 2s - 3ms

In [ ]:
# Function to Build the Attention layer
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1),
        initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1),
        initializer='zeros', trainable=True)
        super(attention, self).build(input_shape)

    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = tfa.tanh(tf.matmul(x,self.W)+self.b)
        # Remove dimension of size 1
        e = tf.squeeze(e, axis=-1)
        # Compute the weights
        alpha = tfa.softmax(e)
        # Reshape to tensorFlow format
        alpha = tf.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = tf.reduce_sum(context, axis=1)
        return context

# Function to Create RNN with the attention layer
def create_RNN_with_attention(hidden_units, dense_units, input_shape, activation):
    x=Input(shape=input_shape)
    RNN_layer = SimpleRNN(hidden_units, return_sequences=True, activation=activation)(x)
    attention_layer = attention()(RNN_layer)
    outputs=Dense(dense_units, trainable=True, activation=activation)(attention_layer)
    model=Model(x,outputs)
    model.compile(loss='mse', optimizer='adam')
    return model

In [ ]:
# 1. Dataset has already been generated

# 2. Model Created: RNN with Attention Layer
model_attention = create_RNN_with_attention(hidden_units=hidden_units, dense_units=1,input_shape=(time_steps,1), activation='tanh')

# 2.1 Visualize:
# model_attention.summary()

# 3. Train
model_attention.fit(trainX, trainY, epochs=epochs, batch_size=1, verbose=2)

# 4. Evaluate
train_mse_attn = model_attention.evaluate(trainX, trainY)
test_mse_attn = model_attention.evaluate(testX, testY)

# 5. Print error
print("Train set MSE with attention = ", train_mse_attn)
print("Test set MSE with attention = ", test_mse_attn)

Epoch 1/30
826/826 - 5s - 6ms/step - loss: 0.0014
Epoch 2/30
826/826 - 2s - 3ms/step - loss: 0.0013
Epoch 3/30
826/826 - 2s - 3ms/step - loss: 0.0012
Epoch 4/30
826/826 - 3s - 3ms/step - loss: 0.0012
Epoch 5/30
826/826 - 2s - 3ms/step - loss: 0.0011
Epoch 6/30
826/826 - 2s - 3ms/step - loss: 0.0011
Epoch 7/30
826/826 - 2s - 3ms/step - loss: 0.0010
Epoch 8/30
826/826 - 2s - 3ms/step - loss: 9.6702e-04
Epoch 9/30
826/826 - 3s - 3ms/step - loss: 9.3256e-04
Epoch 10/30
826/826 - 3s - 3ms/step - loss: 8.7178e-04
Epoch 11/30
826/826 - 2s - 3ms/step - loss: 8.1538e-04
Epoch 12/30
826/826 - 2s - 3ms/step - loss: 7.6302e-04
Epoch 13/30
826/826 - 2s - 3ms/step - loss: 6.8534e-04
Epoch 14/30
826/826 - 2s - 3ms/step - loss: 6.1483e-04
Epoch 15/30
826/826 - 3s - 3ms/step - loss: 5.4419e-04
Epoch 16/30
826/826 - 2s - 3ms/step - loss: 4.7591e-04
Epoch 17/30
826/826 - 2s - 3ms/step - loss: 4.0793e-04
Epoch 18/30
826/826 - 3s - 3ms/step - loss: 3.4287e-04
Epoch 19/30
826/826 - 2s - 3ms/step - loss: 2.8